In [1]:
#TODO: compare time series of large sale to time series of next auction's sales
#TODO: locations
#TODO: regressions, scikit
#TODO: plotly, can explore more
#TODO: standardize date-times with currency exchange rates

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
art_df = pd.read_csv('fina_artframe_2007-2017.csv')
art_df.head()

,Unnamed: 0,artist,auction_name,auction_number,currency,date,high,location,low,piece,sale,day,month,year,adj_sale_price
0,132563,PAUL HUXLEY,sotheby's | auctions - bat iii,AM1105,EUR,4-Oct-11,5000,AMSTERDAM,3000,UNTITLED NO. 33,13181,4,10,2011,14981.03
1,132564,WILLIAM SCOTT,sotheby's | auctions - bat iii,AM1105,EUR,4-Oct-11,60000,AMSTERDAM,40000,CIRCLES DIMINISHING,57930,4,10,2011,65841.06
2,132565,BERNARD COHEN,sotheby's | auctions - bat iii,AM1105,EUR,4-Oct-11,5000,AMSTERDAM,3000,WHEN WHITE,23066,4,10,2011,26215.95
3,132566,ANTONY DONALDSON,sotheby's | auctions - bat iii,AM1105,EUR,4-Oct-11,3000,AMSTERDAM,2000,BRING IT TO JEROME,35786,4,10,2011,40673.02
4,132567,ROBYN DENNY,sotheby's | auctions - bat iii,AM1105,EUR,4-Oct-11,5000,AMSTERDAM,3000,FOR EVER,10709,4,10,2011,12171.45


In [4]:
#Creating 5% sample from the dataframe
art_df_sample = art_df.sample(8700)
art_df_sample


,Unnamed: 0,artist,auction_name,auction_number,currency,date,high,location,low,piece,sale,day,month,year,adj_sale_price
108582,64784,SEBASTIANO RICCI,master paintings: part i,N09302,USD,29-Jan-15,300000,NEW YORK,200000,THE TEMPTATION OF SAINT ANTHONY,0,29,1,2015,0.00
14783,141966,CHEN MAN,sotheby's | auctions - clean air charity auction,HK0374,HKD,4-Apr-11,50000,HONG KONG,30000,ENVIRONMENTAL PROTECTION 83,6172,4,4,2011,7014.86
40170,63944,PABLO PICASSO,impressionist & modern art day sale,L15004,GBP,4-Feb-15,300000,LONDON,200000,TROIS SALTIMBANQUES,336557,4,2,2015,363025.83
153797,196682,HENRI LEBASQUE,sotheby's | auctions - impressionist & modern ...,N08438,USD,8-May-08,120000,NEW YORK,80000,PEUPLIERS EN AUTOMNE PRÈS DE CHALIFERT,103000,8,5,2008,122305.66
72939,176868,"MANNER OF RAFFAELLO SANZIO, CALLED RAPHAEL","sotheby's | auctions - paintings, furniture an...",L09761,GBP,18-Mar-09,3000,LONDON,1500,HERCULES AT THE CROSSROADS,6116,18,3,2009,7288.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135448,142700,JEAN-BAPTISTE-ARMAND GUILLAUMIN,sotheby's | auctions - impressionist & modern ...,N08747,USD,16-Mar-11,30000,NEW YORK,20000,PAYSAGE D'ÎLE DE FRANCE,40625,16,3,2011,46172.85
241,138220,LUCEBERT,sotheby's | auctions - modern & contemporary art,AM1099,EUR,17-May-11,6000,AMSTERDAM,4000,UNTITLED,0,17,5,2011,0.00
68378,160975,ÉMILE OTHON FRIESZ,sotheby's | auctions - impressionist & modern ...,L10005,GBP,4-Feb-10,15000,LONDON,12000,BAIGNEUSES À SAINT-LÉONARD (SARTHE),27754,4,2,2010,32539.95
15663,152015,TANG HAIWEN,sotheby's | auctions - 20th century chinese art,HK0326,HKD,4-Oct-10,50000,HONG KONG,30000,PORTRAIT NO. 50,5155,4,10,2010,6043.94


In [5]:
#finding currencies:
currencies = art_df['currency'].unique()

#separating currencies into their own dataframes
currency_dataframes = {}
for currency in currencies:
    currency_dataframes[currency] = art_df.loc[art_df['currency'] == currency]
currency_dataframes.keys()

usd_df = currency_dataframes['USD']
usd_df.head()

,Unnamed: 0,artist,auction_name,auction_number,currency,date,high,location,low,piece,sale,day,month,year,adj_sale_price
3415,59740,WAEL SHAWKY,contemporary art / doha,DO1501,USD,21-Apr-15,70000,DOHA,50000,THE CAVE: AMSTERDAM,0,21,4,2015,0.00
3416,59741,AHMED MATER,contemporary art / doha,DO1501,USD,21-Apr-15,20000,DOHA,15000,GREEN ANTENNA,22500,21,4,2015,24269.53
3417,59742,CHANT AVEDISSIAN,contemporary art / doha,DO1501,USD,21-Apr-15,250000,DOHA,180000,ICONS OF THE NILE,0,21,4,2015,0.00
3418,59743,KOUR POUR,contemporary art / doha,DO1501,USD,21-Apr-15,90000,DOHA,70000,LOVE CHILD,162500,21,4,2015,175279.96
3419,59744,ALI BANISADR,contemporary art / doha,DO1501,USD,21-Apr-15,150000,DOHA,100000,THE SHRINE,237500,21,4,2015,256178.40


In [6]:
#removing rows that didn't sell
usd_df_sales = usd_df[usd_df['adj_sale_price'] != 0]

#plotting month against the value of the sales
#plt.bar(usd_df_sales['month'], usd_df_sales['adj_sale_price'])


In [31]:
#adding new row describing region
regions = []
for location in art_df['location']:
    if location == ("NEW YORK" or "TORONTO"):
        regions.append("America")
    elif location == ("AMSTERDAM" or "GENEVA" or "LONDON" or "MILAN" or "PARIS" or "ZURICH"):
        regions.append("Europe")
    elif location == ("DOHA" or "DUBAI"):
        regions.append("Middle East")
    elif location == ("MELBOURNE" or "SYDNEY"):
        regions.append("Oceania")
    else:
        regions.append("East Asia")

regions_series = pd.Series(regions)
art_df['region'] = regions_series
art_df.to_csv('art_df_regions.csv')
